# Formulary Lookup Table Creation
### v3.0

Let's start simple and see if we can determine which drugs are in a payer's formulary (reject code for formularies is 70)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('../data/dim_claims.csv')


In [3]:
#Training sample
df_train = df.sample(frac = .75)

#Get list of payers and drugs
payers = df_train['bin'].unique()
drugs = df_train['drug'].unique()

Create a new datafram to hold the rates for all of the different rejection codes (claim acceptance imputed as code -1).

In [5]:
df_code= pd.DataFrame(columns=drugs)

df_code.insert(loc=0,column='Payer',value=0)
df_code.insert(loc=1,column='Acc/Code',value=0)

df_train['reject_code'] = df_train['reject_code'].fillna(-1)
codes = df_train['reject_code'].unique()
rep = len(codes)

p = payers.repeat(rep)

df_code['Payer'] = p

for payer in payers:
    df_code.loc[df_code['Payer'] == payer,['Acc/Code']] = codes
    
df_code.head(20)

,Payer,Acc/Code,B,A,C
0,417740,70.0,NaN,NaN,NaN
1,417740,-1.0,NaN,NaN,NaN
2,417740,76.0,NaN,NaN,NaN
3,417740,75.0,NaN,NaN,NaN
4,417380,70.0,NaN,NaN,NaN
5,417380,-1.0,NaN,NaN,NaN
6,417380,76.0,NaN,NaN,NaN
7,417380,75.0,NaN,NaN,NaN
8,999001,70.0,NaN,NaN,NaN
9,999001,-1.0,NaN,NaN,NaN


Create the table by calculating the approval rates from the training data set.

In [6]:
i = 0

for payer in payers:
    for drug in drugs:
        for code in codes:
            den = df_train[(df_train['bin'] == payer)  & (df_train['drug']==drug)].shape[0]
        #print("Den = ",den)
            if(code>0):
                rej = df_train[(df_train['bin'] == payer)  & (df_train['drug']==drug) 
                               & (df_train['pharmacy_claim_approved']==0) & (df_train['reject_code']==code)].shape[0]
            else:
                rej = 0    
            acc = df_train[(df_train['bin'] == payer)  & (df_train['drug']==drug) & (df_train['pharmacy_claim_approved']==1)].shape[0]
            rejfrac = rej/den
            accfrac = acc/den
            if(code>0):
                df_code.loc[(df_code['Payer'] == payer) & (df_code['Acc/Code']==code), [drug]] = rejfrac
            else:
                df_code.loc[(df_code['Payer'] == payer) & (df_code['Acc/Code']==code), [drug]] = accfrac
        
df_code.head(25)   



,Payer,Acc/Code,B,A,C
0,417740,70.0,1.000000,0.000000,0.000000
1,417740,-1.0,0.000000,0.900768,0.000000
2,417740,76.0,0.000000,0.099232,0.000000
3,417740,75.0,0.000000,0.000000,1.000000
4,417380,70.0,0.000000,0.000000,1.000000
5,417380,-1.0,0.899970,0.000000,0.000000
6,417380,76.0,0.100030,0.000000,0.000000
7,417380,75.0,0.000000,1.000000,0.000000
8,999001,70.0,0.000000,0.000000,0.000000
9,999001,-1.0,0.900851,0.900372,0.898517


#### Conclusions

999001: No drug is rejeced due to formulay or requires prior authorization. All druges are rejected due to code 76

417614: Drug A is not in formulary (always rejected, always due to code 70)
        Drug B is on formulary but does not have preferred status and requires a prior authorization (is always rejected and rejected due to code 75)
        Drug C is accepted most of the time and when rejected it is due to code 76 (that the plan limitations have been exceeded)
        
417740: Drug A is accepted most of the time and when rejected it is due to code 76 (that the plan limitations have been exceeded)
        Drug B is not in formulary (always rejected, always due to code 70)
        Drug C is on formulary but does not have preferred status and requires a prior authorization (is always rejected and rejected due to code 75)
        
417380: Drug A is on formulary but does not have preferred status and requires a prior authorization (is always rejected and rejected due to code 75)
        Drug B is accepted most of the time and when rejected it is due to code 76 (that the plan limitations have been exceeded)
        Drug C is not in formulary (always rejected, always due to code 70)

In [7]:
df_code.to_csv("formulary.csv",index=False) #Export to csv file for use as lookup table later